In [2]:
class Classifier():
    
    import sys
    if '..' not in sys.path:
        sys.path.append('..')
    
    def __init__(self, vectorizer, classifier):
        
        self._load_classifier(classifier)
        self._load_vectorizer(vectorizer)
        
    def _load_classifier(self, classifier):
        # load pre-fitted classifier
        
        from src.multinomialNB import MultiNB
        import pickle
        
        with open(classifier, 'rb') as pf:
            self.classifier = pickle.load(pf)
    
    def _load_vectorizer(self, vectorizer):
        # load pre-fitted vectorizer
        
        from src.Vectorizer import TFIDF_Vectorizer
        import pickle
        
        with open(vectorizer, 'rb') as pf:
            self.vectorizer = pickle.load(pf)
        #self.vectorizer = vectorizer

In [3]:
class DataController():

    import sys
    if '..' not in sys.path:
        sys.path.append('..')
    
    ## init will create dataMatrix
    def __init__(self, pathToFile, num_pool=30, chunk=50):
        import os
        import json
        import pandas as pd
        from multiprocessing import Pool
        from tqdm import tqdm_notebook

        print('loading data')
        loaded_data = []
        with open(pathToFile, 'r', encoding='utf-8') as fin:
            for line in fin:
                loaded_data.append(json.loads(line, encoding='utf-8'))
        
        docs_tokens = []
        print('pre-processing')
        pbar = tqdm_notebook(total=int(len(loaded_data)))
        with Pool(num_pool) as pool:
            pool_result = pool.imap(self._preprocess, loaded_data, chunk)
            for item in pool_result:
                docs_tokens.append(item)
                pbar.update()
        pbar.close()
        
        self.dataSet = pd.DataFrame(docs_tokens, columns=['job_id', 'title','desc','tag','title_seg','desc_seg','sampled'])
        
    def sample(self, size='ALL', repeated=False):
        
        if repeated:
            sample_source = self.dataSet
        else:
            sample_source = self.dataSet.loc[self.dataSet['sampled'] == False]
        if size=='ALL':
            return self.dataSet
        elif type(size) is float:
            size = int(self.dataSet.shape[0]*size)
        size = min(size, sample_source.shape[0])
        ret_sample = sample_source.sample(n=size)
        if not repeated:
            self.dataSet.loc[ret_sample.index, 'sampled'] = True
        #self.dataSet = self.dataSet.drop(ret_sample.index)
        return ret_sample
    
    def _preprocess(self, doc_dict):
        from src import says he doesn’t have gambling debts, and mentions those Nationals tickets ... againtokenizer
        import tltk
        import hashlib
        import time
        def tltk_tokenize(text):
            ret = tltk.segment(text).replace('<u/>', '').replace('<s/>', '').split('|')
            return ret
        cleaner = tokenizer.cleanerFactory("../Resource/charset")
        title = doc_dict['title']
        desc = doc_dict['desc']
        title_seg = tokenizer.tokenize(title, tltk_tokenize, 5, cleaner)
        desc_seg = tokenizer.tokenize(desc, tltk_tokenize, 5, cleaner)
        tag = doc_dict['tag']
        in_str = str(time.time()) + title + desc
        job_id = hashlib.md5(bytes(in_str, 'utf-8')).hexdigest()
        return [job_id, title, desc, tag, title_seg, desc_seg, False]
    
    def add_data(self, data):
        self.dataSet.add(data)
        
    def predict(self, classifier, vectorizer, sample='ALL', repeated=False):
        
        from scipy.sparse import hstack
        import pandas
        
        training_sample = self.sample(sample, repeated)
        prediction = training_sample
        training_Desc = training_sample['desc_seg']
        training_Title = training_sample['title_seg']
        training_Label = training_sample['tag']
        
        #define vectorizer for title and description
        desc_vectorizer = vectorizer.vectorize_desc
        title_vectorizer = vectorizer.vectorize_title
        #transform-vectorize
        title_vec = title_vectorizer.transform(training_Title)
        desc_vec = desc_vectorizer.transform(training_Desc)
        #stack title onto desc
        data_vec = hstack([title_vec, desc_vec])
        #predict class
        label_predict = classifier.predict(data_vec)
        #add prediction to pandas dataframe
        prediction['predict'] = pandas.Series(label_predict, index=prediction.index)
        
        return prediction
    
    def validate_prediction(self, classifier, vectorizer, sample='ALL', repeated=False):
        prediction = self.predict(classifier, vectorizer, sample, repeated)
        false_prediction = prediction.loc[prediction['tag']!=prediction['predict']]
        return false_prediction

In [3]:
#no warning
import warnings
warnings.filterwarnings('ignore')
#init classifier
#import sys
#import pickle
#if '..' not in sys.path:
#    sys.path.append('..')
#from main_fit_vectorizer import TFIDF_Vectorizer
#with open('../Resource/vectorizer_v03.pck', 'rb') as pf:
#    vectorizer = pickle.load(pf)
cClassifier = Classifier('../Resource/vectorizer_v03.pck', '../Resource/MultiNB_STEMvsNONSTEM_0030vs0070_v02.pck')
#end-addhoc-fix
DataSet = DataController('../data/masterDB_JPA_Data_-_20180406_flatten.json', 31, 100)

loading data
pre-processing


In [11]:
import pickle
with open('../Resource/dataset_0822.pck', 'rb') as f:
    DataSet = pickle.load(f)
cClassifier = Classifier('../Resource/vectorizer_v03.pck', '../Resource/MultiNB_STEMvsNONSTEM_0030vs0070_v02.pck')
predicted = DataSet.predict(cClassifier.classifier, cClassifier.vectorizer, 2000, False)
predicted = predicted[['title', 'desc', 'tag', 'predict']]
predicted.to_pickle('../Resource/predicted_STEMnonSTEM_2k_4k.pck')
import pickle
with open('../Resource/dataset_0822.pck', 'wb') as f:
    pickle.dump(DataSet, f)

In [12]:
import pandas
data1 = pandas.read_pickle('../Resource/predicted_STEMnonSTEM_2k_4k.pck')
data2 = pandas.read_pickle('../Resource/predicted_STEMnonSTEM_2k.pck')

In [14]:
data2

,title,desc,tag,predict
0,VP. Business Development (Energy) งานขาย งานบร...,"Location: Thailand, Bangkok Responsibilities: ...",,1
1,International Business Marketing Manager งานขา...,Position Summary: We are seeking a dynamic Int...,,0
2,General Affair Assistant Manager (30K-40K) (JO...,Work outline: Overall of General Affair tasks....,,0
3,General Manager (Factory Manager) @Chonburi (...,Work outline: Be responsible for managing Prod...,,1
4,Sales Coordinator (Retail division) งานธุรการ ...,"Responsibilities: Data entry: Order, Credit N...",,0
5,Senior Architect Manager งานก่อสร้าง > งานสถาป...,"Salary: Negotiable Location: BTS, Bangkok Jo...",,1
6,Technical Engineer (Degree in Electrical/ Mech...,Industry Summary: The Global Leading in Engine...,,1
7,Factory Manager งานบริหาร > งานบริหารทั่วไป งา...,Mai Industrial - Factory Manager A Factory Man...,,1
8,"VP, Legal (300K + Car) - Energy experience onl...",Main duties include: Managing external lawyer...,,0
9,Merchandiser /นักบริหารจัดการสินค้า งานการผลิต...,"รายละเอียดงาน : ประสานงานระหว่าง ภายในโรงงาน,...",,0


In [10]:
DataSet.sample(50, True)

,job_id,title,desc,tag,title_seg,desc_seg,sampled,predict
14191,04a11504b43439a0155d331f6c2c6de2,Process Engineer // Senior Engineer Location:...,Education Required: • B.Eng. or M.Eng. in Mech...,,process|engineer|senior|engineer|location|mpt|...,education|required|b|.|eng|.|or|m|.|eng|.|in|m...,False,1
79106,5bd9374b4b94217439ceefa18249c1ba,ผู้ช่วยพ่อครัว ผู้ช่วยพ่อครัว,- เพศชาย/หญิง- สัญชาติไทย - มีความรับผิดชอบสูง...,,ผู้ช่วย|พ่อครัว|ผู้ช่วยพ่อครัว|ผู้ช่วย|พ่อครัว...,|เพศ|ชาย||เพศชาย||สัญชาติ|ไทย||สัญชาติไทย|-|มี...,False,0
53903,95bade6d4df097a8c1518fc0d67d9ed4,Programmer (Website),"เพศ : ชาย , หญิง|อายุ(ปี) : 25 - 35|ระดับการศึ...",,programmer|website,เพศ|ชาย||หญิง||ปี||-|ระดับ|การศึกษา||ระดับการศ...,False,1
110346,ca128f8462b6f6f20ef8c0c10b3f1c9b,เจ้าหน้าที่ลูกค้าสัมพันธ์/Call Center,1. - ชาย/หญิง อายุ 22 – 35 ปี ปริญญาตรี ทุกสาข...,,เจ้าหน้าที่|ลูกค้า|สัมพันธ์|Call||เจ้าหน้าที่ล...,.|-|ชาย||หญิง|||ปี|||ทุก|สาขา|ทุกสาขา|.|-|มี|ป...,False,0
92706,2de4cc9e55bb80bba3fd17289a5f6002,งานประจำ (Full Time),เดินสายไฟตามแบบ ประกอบ Control box ตามคำสั่ง,,งาน|ประจำ||งานประจำ|full|time,เดิน|สายไฟ|ตาม|แบบ||เดินสายไฟ|สายไฟตาม|ตามแบบ|...,False,1
41002,cfaf69ed499927928405f06e445559e1,เจ้าหน้าที่ธุรการ (ด่วนมาก),"เพศ : ชาย , หญิง|อายุ(ปี) : 18 ปีขึ้นไป|ระดับก...",,เจ้าหน้าที่|ธุร|การ||เจ้าหน้าที่ธุร|ธุรการ|เจ้...,เพศ|ชาย||หญิง||ปี||ปี|ขึ้น|ไป||ปีขึ้น|ขึ้นไป|ป...,False,0
24383,59ebf2516e6537d734ccf91328473a02,แคชเชียร์ สาขา OPPA ม.ธรรมศาสตร์ แคชเชียร์ สาข...,1. อายุ 18 ปีขึ้นไป 2. มีความรอบรอบในการทำงาน ...,,แคชเชียร์|สาขา|oppa|ม|.||แคชเชียร์|สาขา|oppa|ม|.|,.||ปี|ขึ้น|ไป||ปีขึ้น|ขึ้นไป|ปีขึ้นไป|.|มี|ควา...,False,1
16199,05b7697e54fc5da51c1a1e38eeaa056c,ผู้จัดการสาขา (จันทบุรี) ผู้จัดการสาขา (จันทบุรี),• จบการศึกษาวุฒิปริญญาตรี สาขาบริหารธุรกิจ สาข...,,ผู้จัดการ|สาขา|ผู้จัดการสาขา|จันทบุรี|ผู้จัดกา...,||สาขา|บริหาร|ธุรกิจ|สาขาบริหาร|บริหารธุรกิจ|ส...,False,0
46317,0722ec0cbf74c841f3aafe2e4be3d244,Sales Engineer PT18032712,เพศ : ชาย|อายุ(ปี) : 30 - 35 ปีขึ้นไป|ระดับการ...,,sales|engineer|pt18032712,เพศ|ชาย|||ปี||-|ปี|ขึ้น|ไป||ปีขึ้น|ขึ้นไป|ปีขึ...,False,1
96820,1a687dc77f9ce1bdd0d37c1dc5e78a2e,Sales Coordinator (Electronic Part) / 17-25K /...,"Female, Age 25-32 years. Bachelor Degree or re...",,sales|coordinator|electronic|part|17-25k|sathon|p,female|age|25-32|year|.|bachelor|degree|or|rel...,False,1


In [1]:
from src.PredictPage import PredictPage
PredictPage(data=)